In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\DL_OPS\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Projects\\DL_OPS'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Configuration"""
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.CnnClassifer.constants import *
from src.CnnClassifer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion 

        create_directories([config.root_dir]) 
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir 

        )

        return data_ingestion_config

In [11]:
import os 
import urllib.request as request 
import zipfile 
from src.CnnClassifer import logger 
from src.CnnClassifer.utils.common import get_size 

In [12]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}") 
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True) 
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 
except Exception as e :
    raise e 

✅ YAML Content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/khemu003/Dl_ops_project/raw/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion/'}}
[2025-08-06 16:17:51,408: INFO: common: YAML file config\config.yaml loaded successfully]
✅ YAML Content: {'EPOCHS': 1}
[2025-08-06 16:17:51,413: INFO: common: YAML file params.yaml loaded successfully]
[2025-08-06 16:17:51,416: INFO: common: created directory at: artifacts]
[2025-08-06 16:17:51,417: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-06 16:17:54,290: INFO: 144590454: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa